# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
# PART 1: DATA LOADING AND INITIAL EXPLORATION

print("Loading data files...")
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)
print(f"Portfolio: {portfolio.shape[0]} offers")
print(f"Profile: {profile.shape[0]} customers")
print(f"Transcript: {transcript.shape[0]} events")

Loading data files...

DATASET OVERVIEW
Portfolio: 10 offers
Profile: 17000 customers
Transcript: 306534 events


In [4]:
# PART 2: DATA CLEANING

print("\n" + "="*50)
print("DATA CLEANING")
print("="*50)

# Clean Profile Data
print("\nCleaning Profile Data...")
print(f"Missing income values: {profile['income'].isna().sum()}")
print(f"Missing gender values: {profile['gender'].isna().sum()}")
print(f"Age = 118 (missing): {(profile['age'] == 118).sum()}")

# Remove rows with missing demographic data
profile_clean = profile[
    (profile['age'] != 118) & 
    (profile['income'].notna()) & 
    (profile['gender'].notna())
].copy()

print(f"Customers after cleaning: {len(profile_clean)} (removed {len(profile) - len(profile_clean)})")

# Create age groups
profile_clean['age_group'] = pd.cut(
    profile_clean['age'], 
    bins=[0, 30, 45, 60, 100],
    labels=['18-30', '31-45', '46-60', '60+']
)

# Create income groups
profile_clean['income_group'] = pd.cut(
    profile_clean['income'],
    bins=[0, 50000, 75000, 100000, float('inf')],
    labels=['<50k', '50-75k', '75-100k', '>100k']
)

# Calculate membership tenure (days)
profile_clean['became_member_on'] = pd.to_datetime(
    profile_clean['became_member_on'], 
    format='%Y%m%d'
)
reference_date = profile_clean['became_member_on'].max()
profile_clean['member_days'] = (
    reference_date - profile_clean['became_member_on']
).dt.days

# Clean Portfolio Data
print("\nPortfolio Summary:")
print(portfolio[['offer_type', 'difficulty', 'reward', 'duration']].groupby('offer_type').mean())

# Clean Transcript Data
print("\nExpanding transcript value column...")
# Expand the 'value' dictionary column
transcript_expanded = pd.concat([
    transcript.drop('value', axis=1),
    transcript['value'].apply(pd.Series)
], axis=1)

# Rename columns for clarity
transcript_expanded = transcript_expanded.rename(columns={
    'offer id': 'offer_id',
    'offer_id': 'offer_id_alt'
})

# Consolidate offer_id columns
transcript_expanded['offer_id'] = transcript_expanded['offer_id'].fillna(
    transcript_expanded.get('offer_id_alt', '')
)

print(f"Event types: {transcript_expanded['event'].unique()}")
print(f"Event counts:\n{transcript_expanded['event'].value_counts()}")


DATA CLEANING

Cleaning Profile Data...
Missing income values: 2175
Missing gender values: 2175
Age = 118 (missing): 2175
Customers after cleaning: 14825 (removed 2175)

Portfolio Summary:
               difficulty  reward  duration
offer_type                                 
bogo                 7.50     7.5       6.0
discount            11.75     3.0       8.5
informational        0.00     0.0       3.5

Expanding transcript value column...
Event types: ['offer received' 'offer viewed' 'transaction' 'offer completed']
Event counts:
event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64


In [6]:
# PART 3: FEATURE ENGINEERING - CRITICAL FOR ANALYSIS

print("\n" + "="*50)
print("FEATURE ENGINEERING")
print("="*50)

# Key insight: We need to track if customers VIEWED offers before completing them
# This distinguishes influenced purchases from natural purchases

# Separate events by type
offers_received = transcript_expanded[transcript_expanded['event'] == 'offer received'].copy()
offers_viewed = transcript_expanded[transcript_expanded['event'] == 'offer viewed'].copy()
offers_completed = transcript_expanded[transcript_expanded['event'] == 'offer completed'].copy()
transactions = transcript_expanded[transcript_expanded['event'] == 'transaction'].copy()

print(f"\nOffers Received: {len(offers_received)}")
print(f"Offers Viewed: {len(offers_viewed)}")
print(f"Offers Completed: {len(offers_completed)}")
print(f"Transactions: {len(transactions)}")

# Create a function to track offer influence
def analyze_offer_effectiveness(transcript_df, portfolio_df, profile_df):
    """
    Analyze which offers were actually effective (viewed before completion)
    vs accidental completions (completed without viewing)
    """
    
    # First, merge transcript with portfolio to get offer details
    transcript_with_offers = transcript_df.merge(
        portfolio_df[['id', 'offer_type', 'difficulty', 'reward', 'duration']], 
        left_on='offer_id', 
        right_on='id', 
        how='left',
        suffixes=('', '_portfolio')
    )
    
    results = []
    
    # Group by person
    for person_id in transcript_with_offers['person'].unique():
        person_events = transcript_with_offers[
            transcript_with_offers['person'] == person_id
        ].sort_values('time')
        
        # Track active offers for this person
        active_offers = {}  # {offer_id: {'received_time': X, 'viewed': False, 'valid_until': Y}}
        
        for _, event in person_events.iterrows():
            event_type = event['event']
            event_time = event['time']
            
            if event_type == 'offer received':
                offer_id = event['offer_id']
                # Check if we have portfolio data for this offer
                if pd.notna(event.get('duration')):
                    duration = event['duration']
                    active_offers[offer_id] = {
                        'received_time': event_time,
                        'viewed': False,
                        'valid_until': event_time + (duration * 24),  # Convert days to hours
                        'offer_type': event['offer_type'],
                        'difficulty': event['difficulty'],
                        'reward': event['reward']
                    }
            
            elif event_type == 'offer viewed':
                offer_id = event['offer_id']
                if offer_id in active_offers:
                    active_offers[offer_id]['viewed'] = True
            
            elif event_type == 'offer completed':
                offer_id = event['offer_id']
                if offer_id in active_offers:
                    offer_info = active_offers[offer_id]
                    
                    # Check if offer was viewed (influenced) or not
                    results.append({
                        'person': person_id,
                        'offer_id': offer_id,
                        'offer_type': offer_info['offer_type'],
                        'difficulty': offer_info['difficulty'],
                        'reward': offer_info['reward'],
                        'was_viewed': offer_info['viewed'],
                        'completion_time': event_time - offer_info['received_time']
                    })
                    
                    # Remove completed offer
                    del active_offers[offer_id]
    
    return pd.DataFrame(results)

print("\nAnalyzing offer effectiveness (this may take a moment)...")
offer_analysis = analyze_offer_effectiveness(transcript_expanded, portfolio, profile_clean)

# Also merge with portfolio for later use
offers_received = offers_received.merge(
    portfolio[['id', 'offer_type', 'difficulty', 'reward', 'duration']], 
    left_on='offer_id', 
    right_on='id', 
    how='left'
)

print(f"\nTotal offer completions analyzed: {len(offer_analysis)}")
print(f"Completions where offer was VIEWED (influenced): {offer_analysis['was_viewed'].sum()}")
print(f"Completions where offer was NOT viewed (accidental): {(~offer_analysis['was_viewed']).sum()}")
print(f"Influence rate: {offer_analysis['was_viewed'].mean():.1%}")



FEATURE ENGINEERING

Offers Received: 76277
Offers Viewed: 57725
Offers Completed: 33579
Transactions: 138953

Analyzing offer effectiveness (this may take a moment)...

Total offer completions analyzed: 32757
Completions where offer was VIEWED (influenced): 22383
Completions where offer was NOT viewed (accidental): 10374
Influence rate: 68.3%


In [7]:
# PART 4: DEMOGRAPHIC ANALYSIS

print("\n" + "="*50)
print("DEMOGRAPHIC RESPONSE ANALYSIS")
print("="*50)

# Merge offer analysis with customer demographics
offer_demo = offer_analysis.merge(
    profile_clean[['id', 'age', 'gender', 'income', 'age_group', 'income_group', 'member_days']],
    left_on='person',
    right_on='id',
    how='left'
)

# Filter only influenced completions (offers that were viewed)
influenced_offers = offer_demo[offer_demo['was_viewed'] == True].copy()

print("\nOffer Response by Demographics (Influenced Only):")
print("\n1. By Gender:")
gender_response = influenced_offers.groupby(['gender', 'offer_type']).size().unstack(fill_value=0)
print(gender_response)
print("\nPercentage by gender:")
print(gender_response.div(gender_response.sum(axis=1), axis=0).round(3))

print("\n2. By Income Group:")
income_response = influenced_offers.groupby(['income_group', 'offer_type']).size().unstack(fill_value=0)
print(income_response)
print("\nPercentage by income:")
print(income_response.div(income_response.sum(axis=1), axis=0).round(3))

print("\n3. By Age Group:")
age_response = influenced_offers.groupby(['age_group', 'offer_type']).size().unstack(fill_value=0)
print(age_response)
print("\nPercentage by age:")
print(age_response.div(age_response.sum(axis=1), axis=0).round(3))


DEMOGRAPHIC RESPONSE ANALYSIS

Offer Response by Demographics (Influenced Only):

1. By Gender:
offer_type  bogo  discount
gender                    
F           5024      5015
M           5020      6054
O            179       186

Percentage by gender:
offer_type   bogo  discount
gender                     
F           0.500     0.500
M           0.453     0.547
O           0.490     0.510

2. By Income Group:
offer_type    bogo  discount
income_group                
<50k          1843      2392
50-75k        4552      5076
75-100k       2969      3036
>100k          859       751

Percentage by income:
offer_type     bogo  discount
income_group                 
<50k          0.435     0.565
50-75k        0.473     0.527
75-100k       0.494     0.506
>100k         0.534     0.466

3. By Age Group:
offer_type  bogo  discount
age_group                 
18-30        811      1041
31-45       1761      2002
46-60       3593      3886
60+         4056      4320

Percentage by age:
offer_t

/tmp/ipykernel_13/2679451462.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  income_response = influenced_offers.groupby(['income_group', 'offer_type']).size().unstack(fill_value=0)
/tmp/ipykernel_13/2679451462.py:32: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_response = influenced_offers.groupby(['age_group', 'offer_type']).size().unstack(fill_value=0)


In [9]:
# PART 5: CALCULATE CONVERSION RATES

print("\n" + "="*50)
print("CONVERSION RATE ANALYSIS")
print("="*50)

# Calculate rates: received -> viewed -> completed
received_counts = offers_received.groupby('offer_type').size()
viewed_counts = offers_viewed.merge(
    portfolio[['id', 'offer_type']], 
    left_on='offer_id', 
    right_on='id'
).groupby('offer_type').size()
completed_viewed_counts = influenced_offers.groupby('offer_type').size()

conversion_summary = pd.DataFrame({
    'Received': received_counts,
    'Viewed': viewed_counts,
    'Completed (Viewed)': completed_viewed_counts
})

conversion_summary['View_Rate'] = (
    conversion_summary['Viewed'] / conversion_summary['Received']
).round(3)
conversion_summary['Completion_Rate'] = (
    conversion_summary['Completed (Viewed)'] / conversion_summary['Viewed']
).round(3)
conversion_summary['Overall_Success_Rate'] = (
    conversion_summary['Completed (Viewed)'] / conversion_summary['Received']
).round(3)

print("\nConversion Funnel by Offer Type:")
print(conversion_summary)



CONVERSION RATE ANALYSIS

Conversion Funnel by Offer Type:
               Received  Viewed  Completed (Viewed)  View_Rate  \
offer_type                                                       
bogo              30499   25449             10506.0      0.834   
discount          30543   21445             11877.0      0.702   
informational     15235   10831                 NaN      0.711   

               Completion_Rate  Overall_Success_Rate  
offer_type                                            
bogo                     0.413                 0.344  
discount                 0.554                 0.389  
informational              NaN                   NaN  


In [10]:
# PART 6: TRANSACTION ANALYSIS

print("\n" + "="*50)
print("TRANSACTION & REVENUE ANALYSIS")
print("="*50)

# Merge transactions with demographics
transactions_demo = transactions.merge(
    profile_clean[['id', 'gender', 'income_group', 'age_group']],
    left_on='person',
    right_on='id',
    how='left'
)

print("\nAverage Transaction Amount by Demographics:")
print(f"Overall: ${transactions_demo['amount'].mean():.2f}")
print(f"\nBy Gender:\n{transactions_demo.groupby('gender')['amount'].mean().round(2)}")
print(f"\nBy Income Group:\n{transactions_demo.groupby('income_group')['amount'].mean().round(2)}")
print(f"\nBy Age Group:\n{transactions_demo.groupby('age_group')['amount'].mean().round(2)}")

# Calculate revenue from completed offers
print("\nRevenue from Influenced Offers:")
total_reward = influenced_offers['reward'].sum()
print(f"Total rewards given: ${total_reward:.2f}")
print(f"Average reward per completion: ${influenced_offers['reward'].mean():.2f}")



TRANSACTION & REVENUE ANALYSIS

Average Transaction Amount by Demographics:
Overall: $12.78

By Gender:
gender
F    17.49
M    11.61
O    14.80
Name: amount, dtype: float64

By Income Group:
income_group
<50k        6.83
50-75k     12.64
75-100k    26.39
>100k      30.90
Name: amount, dtype: float64

By Age Group:
age_group
18-30     8.23
31-45    11.08
46-60    15.75
60+      16.47
Name: amount, dtype: float64

Revenue from Influenced Offers:
Total rewards given: $0.00
Average reward per completion: $nan


/tmp/ipykernel_13/2895816913.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(f"\nBy Income Group:\n{transactions_demo.groupby('income_group')['amount'].mean().round(2)}")
/tmp/ipykernel_13/2895816913.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(f"\nBy Age Group:\n{transactions_demo.groupby('age_group')['amount'].mean().round(2)}")


In [11]:
# PART 7: KEY INSIGHTS & RECOMMENDATIONS

print("\n" + "="*50)
print("KEY INSIGHTS & RECOMMENDATIONS")
print("="*50)

print("\n1. OFFER EFFECTIVENESS:")
print(f"   - Only {offer_analysis['was_viewed'].mean():.1%} of completions were influenced by viewing")
print(f"   - This means customers often complete offers without seeing them")
print(f"   - Focus on offers with high view rates to ensure influence")

print("\n2. BEST PERFORMING OFFER TYPES:")
best_offer = conversion_summary['Overall_Success_Rate'].idxmax()
print(f"   - {best_offer} has highest overall success rate: "
      f"{conversion_summary.loc[best_offer, 'Overall_Success_Rate']:.1%}")

print("\n3. DEMOGRAPHIC TARGETING:")
for offer_type in influenced_offers['offer_type'].unique():
    offer_subset = influenced_offers[influenced_offers['offer_type'] == offer_type]
    
    if len(offer_subset) > 0:
        top_gender = offer_subset['gender'].mode().values[0] if len(offer_subset['gender'].mode()) > 0 else 'N/A'
        top_income = offer_subset['income_group'].mode().values[0] if len(offer_subset['income_group'].mode()) > 0 else 'N/A'
        top_age = offer_subset['age_group'].mode().values[0] if len(offer_subset['age_group'].mode()) > 0 else 'N/A'
        
        print(f"\n   {offer_type.upper()}:")
        print(f"   - Most responsive: Gender={top_gender}, Income={top_income}, Age={top_age}")

print("\n4. OPTIMIZATION OPPORTUNITIES:")
print("   - Send fewer offers to customers who complete without viewing")
print("   - Focus marketing spend on customers who engage with offers")
print("   - Personalize offer difficulty based on typical transaction amounts")

print("\n" + "="*50)
print("ANALYSIS COMPLETE")
print("="*50)
print("\nNext Steps:")
print("1. Build predictive model for offer response")
print("2. Calculate ROI for each offer type")
print("3. Create customer segments for targeted marketing")
print("4. A/B test new offer strategies")



KEY INSIGHTS & RECOMMENDATIONS

1. OFFER EFFECTIVENESS:
   - Only 68.3% of completions were influenced by viewing
   - This means customers often complete offers without seeing them
   - Focus on offers with high view rates to ensure influence

2. BEST PERFORMING OFFER TYPES:
   - discount has highest overall success rate: 38.9%

3. DEMOGRAPHIC TARGETING:

   BOGO:
   - Most responsive: Gender=F, Income=50-75k, Age=60+

   DISCOUNT:
   - Most responsive: Gender=M, Income=50-75k, Age=60+

4. OPTIMIZATION OPPORTUNITIES:
   - Send fewer offers to customers who complete without viewing
   - Focus marketing spend on customers who engage with offers
   - Personalize offer difficulty based on typical transaction amounts

ANALYSIS COMPLETE

Next Steps:
1. Build predictive model for offer response
2. Calculate ROI for each offer type
3. Create customer segments for targeted marketing
4. A/B test new offer strategies


In [12]:
# OPTIONAL: SAVE CLEANED DATA FOR FURTHER ANALYSIS

print("\nSaving cleaned datasets...")
profile_clean.to_csv('profile_clean.csv', index=False)
offer_demo.to_csv('offer_analysis_with_demographics.csv', index=False)
print("Files saved: profile_clean.csv, offer_analysis_with_demographics.csv")


Saving cleaned datasets...
Files saved: profile_clean.csv, offer_analysis_with_demographics.csv
